In [1]:
#!/usr/bin/env python
# coding: utf-8


# Dependencies and Setup
import pandas as pd
import numpy as np
from IPython.display import display

# surpress false positive warnings
pd.options.mode.chained_assignment = None  # default='warn'

# File to Load (Remember to Change These)
schoolDataToLoad = "Resources/schools_complete.csv"
studentDataToLoad = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
schoolData = pd.read_csv(schoolDataToLoad)
studentData = pd.read_csv(studentDataToLoad)
# Combine the data into a single dataset
schoolDataComplete = pd.merge(studentData, schoolData, how="left", on=["school_name", "school_name"])

schoolNames = schoolData["school_name"]



In [57]:
def summarizeDistrict(schoolDataComplete):
    totalSchools = len(schoolDataComplete['school_name'].unique())
    totalStudents = schoolDataComplete.shape[0]
    totalBudget = schoolData['budget'].sum()
    totalMathPass = dict((schoolDataComplete["math_score"]>=70).value_counts())[True]
    totalReadingPass = dict((schoolDataComplete["reading_score"]>=70).value_counts())[True]
    averageMathScore = schoolDataComplete['math_score'].mean()
    averageReadingScore = schoolDataComplete['reading_score'].mean()
    percentMathPass = round(totalMathPass/totalStudents*100, 6)
    percentReadingPass = round(totalReadingPass/totalStudents*100, 6)

    districtSummary = pd.DataFrame(
        {'Total Schools' : [totalSchools],
        'Total Students' : [totalStudents],
        'Total Budget' : ["${0:,.2f}".format(totalBudget)],
        'Per Student Budget' : ["${0:,.2f}".format(totalBudget/totalStudents)],
        'Average Math Score' : [round(averageMathScore, 6)],
        'Average Reading Score': [round(averageReadingScore,6)],
        '% Students Pass Math' : [percentMathPass],
        '% Students Pass Reading' : [percentReadingPass],
        'Average Pass Rate' : [(percentMathPass+percentReadingPass)/6]})

    return districtSummary

In [56]:
def summarizeSchools(schoolDataComplete):
    schools = []
    for highSchool in schoolNames:
        thisSchoolTable = schoolDataComplete[schoolDataComplete['school_name'] == highSchool]
        _, _, _, _, schoolName, _, _, id, type, students, budget = thisSchoolTable.iloc[0]
        totalMathPass = dict((thisSchoolTable["math_score"]>=70).value_counts())[True]
        totalReadingPass = dict((thisSchoolTable["reading_score"]>=70).value_counts())[True]
        percentMathPass = round(totalMathPass/students*100, 6)
        percentReadingPass = round(totalReadingPass/students*100, 6)

        schools.append (
            {'School Name' : schoolName,
            'School Type' : type,
            'Total Students' : students,
            'Total School Budget' : "${0:,.2f}".format(budget),
            'Per Student Budget' : "${0:,.2f}".format(budget/students),
            'Average Math Score' : round(thisSchoolTable['math_score'].mean(), 6),
            'Average Reading Score' : round(thisSchoolTable['reading_score'].mean(), 6),
            '% Passing Math' : percentMathPass,
            '% Passing Reading' : percentReadingPass,
            'Overall Passing Rate': (percentMathPass + percentReadingPass)/2
            })
    schoolsSummary = pd.DataFrame (schools)
    schoolsSummary = schoolsSummary[['School Name', 'School Type', "Total Students", 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]
    return schoolsSummary


In [18]:
def summarizeScoresByGrade (schoolDataComplete, subject):
    # subject: "math" or "reading"
    subjectScore = subject+"_score"
    
    # a series per grade
    series9thGrade = schoolDataComplete.loc[schoolDataComplete["grade"]=="9th"]
    series10thGrade = schoolDataComplete.loc[schoolDataComplete["grade"]=="10th"]
    series11thGrade = schoolDataComplete.loc[schoolDataComplete["grade"]=="11th"]
    series12thGrade = schoolDataComplete.loc[schoolDataComplete["grade"]=="12th"]

    # group the series by school and calculate average
    series9thGradeBySchool = series9thGrade.groupby("school_name", as_index=False)[subjectScore].mean().round(6)
    series10thGradeBySchool = series10thGrade.groupby("school_name", as_index=False)[subjectScore].mean().round(6)
    series11thGradeBySchool = series11thGrade.groupby("school_name", as_index=False)[subjectScore].mean().round(6)
    series12thGradeBySchool = series12thGrade.groupby("school_name", as_index=False)[subjectScore].mean().round(6)
    
    # merge all series on school name
    allGradesBySchool = pd.merge(series9thGradeBySchool, series10thGradeBySchool, on="school_name").merge(series11thGradeBySchool, on="school_name").merge(series12thGradeBySchool, on="school_name")
    # rename columns
    allGradesBySchool.columns = ["School Name", "9th", "10th", "11th", "12th"]
    allGradesBySchool = allGradesBySchool.set_index('School Name')
    # remove index name row for cleaner output
    allGradesBySchool.index.name = None
    return (allGradesBySchool)



In [19]:
def sortNumSchools (by, howMany, order):
    # Top Performing schools
    if order == 'descending':
        sortedSchoolsSummary = schoolsSummary.sort_values([by], ascending=False)
    # Bottom Performing schools
    else:
        sortedSchoolsSummary = schoolsSummary.sort_values([by], ascending=True)
    sortedSchoolsSummary = sortedSchoolsSummary.set_index('School Name')
    sortedSchoolsSummary.index.name = None
    return sortedSchoolsSummary.head(howMany)

In [20]:
def summarizeScoresBySpendingBudget(schoolsSummary):
    bins=[0, 585, 615, 645, 675]
    binLabels = ["<$585", "\$585-\$615", "\$615-\$645", "\$645-\$675"]
    #convert per student budget back to float for binning
    spendingRanges = pd.cut(schoolsSummary["Per Student Budget"].replace("\$", "", regex=True).astype(float), bins, labels=binLabels)
    schoolPerfBySpendingRange = schoolsSummary[["School Name", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]
    schoolPerfBySpendingRange ["Spending Ranges (Per Student)"] = spendingRanges

    return (schoolPerfBySpendingRange.sort_values(by="Spending Ranges (Per Student)").groupby("Spending Ranges (Per Student)").mean().round(6))




In [21]:
def summarizeScoresBySchoolSize(schoolsSummary):
    bins = [0, 1000, 2000, 5000]
    binLabels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
    sizeRanges = pd.cut(schoolsSummary["Total Students"], bins, labels=binLabels)
    schoolPerfBySize = schoolsSummary[["School Name", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]
    schoolPerfBySize ["School Size"] = sizeRanges

    return (schoolPerfBySize.sort_values(by="School Size").groupby("School Size").mean().round(6))

In [22]:
def summarizeScoresBySchoolType(schoolsSummary):
    return schoolsSummary[["School Type", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]].sort_values(by="School Type").groupby("School Type").mean().round(6)


In [58]:
districtSummary = summarizeDistrict(schoolDataComplete)
schoolsSummary = summarizeSchools(schoolDataComplete)

## District Summary

In [59]:
districtSummary [['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score','% Students Pass Math', "% Students Pass Reading", 'Average Pass Rate']]

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Students Pass Math,% Students Pass Reading,Average Pass Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,26.797719


## School Summary 

In [60]:
schoolsSummary [['School Name', 'Per Student Budget', 'Total Students', 'Overall Passing Rate']]

,School Name,Per Student Budget,Total Students,Overall Passing Rate
0,Huang High School,$655.00,2917,73.500171
1,Figueroa High School,$639.00,2949,73.363853
2,Shelton High School,$600.00,1761,94.860874
3,Hernandez High School,$652.00,4635,73.807983
4,Griffin High School,$625.00,1468,95.265668
5,Wilson High School,$578.00,2283,95.203679
6,Cabrera High School,$582.00,1858,95.586652
7,Bailey High School,$628.00,4976,74.306672
8,Holden High School,$581.00,427,94.379391
9,Pena High School,$609.00,962,95.270271


## Top Performing Schools (by Passing Rate)


In [61]:
sortNumSchools ("Overall Passing Rate", 5, "descending")

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270271
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (by Passing Rate)

In [62]:
sortNumSchools("Overall Passing Rate", 5, "ascending")

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363853
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Score by Grade

In [63]:
summarizeScoresByGrade (schoolDataComplete, "math")

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade

In [64]:
summarizeScoresByGrade (schoolDataComplete, "reading")

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [65]:
summarizeScoresBySpendingBudget(schoolsSummary)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
\$585-\$615,83.599686,83.885212,94.230858,95.900287,95.065572
\$615-\$645,79.079225,81.891436,75.668212,86.106569,80.887391
\$645-\$675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

In [66]:
summarizeScoresBySchoolSize(schoolsSummary)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929844,93.550225,96.099436,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766635,76.364998


## Scores by School Type

In [67]:
summarizeScoresBySchoolType(schoolsSummary)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


## Conclusions
* School performance, as measured by overall student passing rate, was not dependent on the per student budget. In fact, it appears that the higher the spending, the lower the overall passing rate.

* School performance is directly tied to: 1) type of school and 2) school size:

    1) Charter schools have a much better performance (overall passing rate - 95%) than district schools (73%)
    
    2) Schools that have under 2000 students have a much higher passing rate (~95%) than schools that have between 2000-5000 students